In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
from akorn import *
from flh import *
%load_ext autoreload
%autoreload 2

%matplotlib qt

In [3]:
%load_ext autoreload
%autoreload 2
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data



utils = importr('utils')
base = importr('base')
glmgen = importr('glmgen')
%matplotlib qt

std = 0.3
var = std**2

# mse_akorn = []
# mse_tf = []



def doppler(x, epsilon):
    return np.sin(2 * np.pi * (1 + epsilon) / (x + epsilon))

from util import piecewise_linear

print("Done!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done!


In [5]:
mse_akorn = []
mse_addle = []
mse_tf = []
mse_tf_dof = []
# Ts = [100, 5000, 100_000, 200_000, 500_000]
Ts = [50, 100, 200, 500, 1000, 2500, 5000]
# Ts = [50, 100]



# # Ts = [100]
runs_per_T = 20

from util import *

akorn_fv = None
dof_fv = None
tf_fv = None
addle_fv = None
    
for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 
    Xr = robjects.FloatVector(X)

    G = [piecewise_linear([1,-1,3, -3, 12, 0,-12,0,  -3, 1],np.array([0.25, 0.5, 0.6, 0.8, 1.2, 1.21, 1.22, 1.23,  1.7, 1.9])/2, x, 0) for x in X]
    G = G/np.max(G)

    mses_T_tf = []
    mses_T_tf_dof = []
    mses_T_akorn = []
    mses_T_addle = []

    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))

        ak = AKORN(X, Y, var)
        ak.train()
        fv = ak.preds
        
        online = FLH(X, Y, T, lr = 1/8*(np.max(np.abs(Y))**2))
        online.run()
        fv_addle = online.predictions
        
        
        mses_T_addle.append(((fv_addle - G)**2).mean())


        mses_T_akorn.append(((fv - G)**2).mean())


        Yr = robjects.FloatVector(Y)
        tf = glmgen.trendfilter(Xr, Yr, k = 1)
        rcode1 = "predict(%s)" %(tf.r_repr())
        tf_fit = np.array(robjects.r(rcode1)).T

        best_mse = float('inf')
        best_mse_est = float('inf')
        best_mse_dof = float('inf')
        best_tf_fit = None
        best_tf_fit_dof = None
        for t in tf_fit:
            tf_mse = ((t - G)**2).mean() #Oracle risk
            tf_mse_est = (np.sum(((t - Y)**2)) + 2*var*count_linear_pieces(t)) #Stein risk estimate
            if tf_mse < best_mse:
                best_mse = tf_mse
                best_tf_fit = t
            if tf_mse_est < best_mse_est:
                best_mse_est = tf_mse_est
                best_mse_dof = tf_mse
                best_tf_fit_dof = t
                
        mses_T_tf.append(((best_tf_fit - G)**2).mean())
        mses_T_tf_dof.append(((best_tf_fit_dof - G)**2).mean())
        
        akorn_fv = fv
        addle_fv = fv_addle
        tf_dof_fv = best_tf_fit_dof
        tf_fv = best_tf_fit

    #     Z_tf.append(best_tf_fit)
    mse_tf.append(mses_T_tf)
    mse_tf_dof.append(mses_T_tf_dof)
    mse_addle.append(mses_T_addle)
    mse_akorn.append(mses_T_akorn)
    

avg_mses_akorn = [np.array(mse_T).mean() for mse_T in mse_akorn]
avg_mses_addle = [np.array(mse_T).mean() for mse_T in mse_addle]
avg_mses_tf = [np.array(mse_T).mean() for mse_T in mse_tf]
avg_mses_tf_dof = [np.array(mse_T).mean() for mse_T in mse_tf_dof]
print("Done!")

T =  50
Run number 0
lr =  0.032168978989196025
run
0
Run number 1
lr =  0.0281917014888701
run
0
Run number 2
lr =  0.0417975308526615
run
0
Run number 3
lr =  0.05493096673911375
run
0
Run number 4
lr =  0.03482875098766392
run
0
Run number 5
lr =  0.03239546264201818
run
0
Run number 6
lr =  0.03621641492504229
run
0
Run number 7
lr =  0.058657517412666525
run
0
Run number 8
lr =  0.05237021321233079
run
0
Run number 9
lr =  0.045834529288892076
run
0
Run number 10
lr =  0.04805620617784779
run
0
Run number 11
lr =  0.051182177997248245
run
0
Run number 12
lr =  0.06546825940394037
run
0
Run number 13
lr =  0.03891510847127244
run
0
Run number 14
lr =  0.03379853090976556
run
0
Run number 15
lr =  0.047124448441564964
run
0
Run number 16
lr =  0.0648625854476978
run
0
Run number 17
lr =  0.036388071043931415
run
0
Run number 18
lr =  0.04862066414932829
run
0
Run number 19
lr =  0.028112874487632278
run
0
T =  100
Run number 0
lr =  0.04263534408120155
run
0
Run number 1
lr =  0.037

In [22]:
from sklearn.linear_model import LinearRegression
akorn_ref = LinearRegression().fit(np.log(np.array(Ts)[1:]).reshape(-1, 1), np.log(avg_mses_akorn)[1:])
addle_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_addle))
tf_ref = LinearRegression().fit(np.log(np.array(Ts[1:])).reshape(-1, 1), np.log(np.array(avg_mses_tf))[1:])
tf_dof_ref = LinearRegression().fit(np.log(np.array(Ts[1:])).reshape(-1, 1), np.log(np.array(avg_mses_tf_dof)[1:]))

In [31]:
import pywt
def wavelet_denoise(Y, a, std):
        coeffs = pywt.wavedec(Y, "db2")
        thresholded = []
        for c in coeffs:
            thresholded.append(pywt.threshold(c, a*std*math.sqrt(2*math.log(T)), mode = "soft"))
        signal = pywt.waverec(thresholded, "db2")
        return signal
    
def oracle_wavelets(Y,G, std):
    best_fit = None
    best_mse = float('inf')
    best_a = None
    for a in np.linspace(0, 2, 100):
        sig = wavelet_denoise(Y, a, std)
        sig_mse = np.mean(np.square(sig - G))
        if sig_mse < best_mse:
            print("a = ", a)
            best_fit = sig
            best_mse = sig_mse
            best_a = a
    return (best_fit, best_mse, best_a)
mse_wav = []
wav_fv = None
runs_per_T = 20

for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 

    G = [piecewise_linear([1,-1,3, -3, 12, 0,-12,0,  -3, 1],np.array([0.25, 0.5, 0.6, 0.8, 1.2, 1.21, 1.22, 1.23,  1.7, 1.9])/2, x, 0) for x in X]
    G = G/np.max(G)

    mses_T_wav = []
    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))
        
        fv_wav = oracle_wavelets(Y, G, std)[0]
        mses_T_wav.append(((fv_wav - G)**2).mean())
        
        wav_fv =  fv_wav

    #     Z_tf.append(best_tf_fit)
    mse_wav.append(mses_T_wav)
    

avg_mses_wav = [np.array(mse_T).mean() for mse_T in mse_wav]
wav_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_wav)))
print("Done!")

T =  50
Run number 0
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
Run number 1
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a

Run number 6
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
a =  0.4646464646464647
a =  0.48484848484848486
a =  0.5050505050505051
Run number 7
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.282

a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
Run number 8
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.343434343434343

Run number 12
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
a =  0.4646464646464647
a =  0.48484848484848486
Run number 13
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.3

Run number 7
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
a =  0.4646464646464647
a =  0.48484848484848486
a =  0.5050505050505051
a =  0.5252525252525253
Run number 8
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.2626

Run number 0
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
a =  0.4646464646464647
a =  0.48484848484848486
a =  0.5050505050505051
a =  0.5252525252525253
a =  0.5454545454545455
Run number 1
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242

a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
a =  0.4646464646464647
a =  0.48484848484848486
a =  0.5050505050505051
a =  0.5252525252525253
Run number 2
a =  0.0
a =  0.020202020202020204
a =  0.04040404040404041
a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.44444

a =  0.06060606060606061
a =  0.08080808080808081
a =  0.10101010101010102
a =  0.12121212121212122
a =  0.14141414141414144
a =  0.16161616161616163
a =  0.18181818181818182
a =  0.20202020202020204
a =  0.22222222222222224
a =  0.24242424242424243
a =  0.26262626262626265
a =  0.2828282828282829
a =  0.30303030303030304
a =  0.32323232323232326
a =  0.3434343434343435
a =  0.36363636363636365
a =  0.38383838383838387
a =  0.4040404040404041
a =  0.42424242424242425
a =  0.4444444444444445
a =  0.4646464646464647
a =  0.48484848484848486
a =  0.5050505050505051
a =  0.5252525252525253
a =  0.5454545454545455
Done!


In [37]:
def quick_round_str(z):
    return str(round(z, 5))

plt.clf()
from util import subplots_centered
T = Ts[-1]
X = np.linspace(0, T, T)/T 
G = [piecewise_linear([1,-1,3, -3, 12, 0,-12,0,  -3, 1],np.array([0.25, 0.5, 0.6, 0.8, 1.2, 1.21, 1.22, 1.23,  1.7, 1.9])/2, x, 0) for x in X]
G = G/np.max(G)

f, ax = plt.subplots(1, 2)
# f, ax = subplots_
ax0 = ax[0]
ax1 = ax[1]
ax1.plot(range(T), G, color = "black")
ax1.plot(range(T), tf_fv, color = "tab:blue")
ax1.plot(range(T), Y, alpha = 0.15, color = "black")
# ax1.plot(range(T), wav_fv, color = "purple")
ax1.plot(range(T), akorn_fv, color = "tab:orange")
# ax1.plot(range(T), addle_fv, color = "green")

ax1.yaxis.set_visible(False)

# f.suptitle("Rates for PW Linear Function")
ax0.set_xlabel("n")
ax0.set_ylabel("MSE")

# ax0.set_ylim([10**(-4), 10**(-1)])
# ax1.set_ylim([10**(-4), 10**(0)])

ax0.grid(True)



ax0.loglog(Ts, [T**(-4/5) for T in Ts], linestyle = "dashed", label = "$n^{-4/5}$ line", color = "black")
ax0.loglog(Ts, avg_mses_tf, marker = 'o', label = "oracle trend filtering", color = "tab:blue")
ax0.loglog(Ts, avg_mses_wav, marker = 'o', label = "wavelets", color = "purple")
ax0.loglog(Ts, avg_mses_tf_dof, marker = 'o', label = "tf dof", color = "blue")
ax0.loglog(Ts, avg_mses_akorn, marker = 'o', label = "akorn", color = "tab:orange")


f.legend(framealpha = 1)

In [21]:
mses_akorn

NameError: name 'mses_akorn' is not defined